In [1]:
from caveclient import CAVEclient
import pcg_skel
from pcg_skel import features
import meshparty
from meshparty import meshwork
import os
import pandas as  pd
import scipy
print (scipy.__version__)
from ast import literal_eval
from datetime import datetime, timezone
settime3 = datetime(2023, 9, 19, 4, 0, 0, 0, tzinfo=timezone.utc)
client = CAVEclient('minnie65_phase3_v1')

C:\Users\clare.gamlin\AppData\Local\anaconda3\lib\site-packages\python_jsonschema_objects\__init__.py:113: UserWarning: Schema id not specified. Defaulting to 'self'
  warnings.warn("Schema id not specified. Defaulting to 'self'")


1.10.1


In [3]:
###query for all cells in the inhibitory cells in the cortical column and in the MC table
column_cells = client.materialize.query_table('allen_column_mtypes_v1', timestamp = settime3)
inh_col = column_cells.loc[column_cells['classification_system'] == 'inhibitory']
inh_col_simple = inh_col[['pt_root_id','pt_position']]

###query for curated MCs
cg_table= client.materialize.tables.gamlin_2023_mcs().live_query(timestamp=settime3)
cg_table_simple =  cg_table[['pt_root_id','pt_position']]

##concatenate the two dfs together
cg_plus_col = pd.concat([inh_col_simple,cg_table_simple])
# ##identify root ids of duplicated cells
# dup_df = cg_plus_col[cg_plus_col['pt_root_id'].duplicated() == True]
###drop duplicates
cg_plus_nodup = cg_plus_col.drop_duplicates(subset = 'pt_root_id')
###generate list of cell ids of inh column cells and MCs
oids = cg_plus_nodup['pt_root_id'].tolist()
###generate list of soma locations of inh column cells and MCs
roots = cg_plus_nodup['pt_position'].tolist()


The `client.materialize.tables` interface is experimental and might experience breaking changes before the feature is stabilized.


[array([172512, 175280,  21964]), array([179792, 215664,  22864]), array([173360, 159856,  22024]), array([172624, 126960,  20812]), array([195424, 223792,  23343]), array([179792, 179488,  20946]), array([191120, 109792,  21792]), array([185920, 257680,  22934]), array([196288, 247568,  24687]), array([180752, 135200,  22612]), array([177936, 215664,  21555]), array([181680, 160112,  21810]), array([178176, 152192,  20167]), array([189040, 118352,  21048]), array([168512, 127088,  20650]), array([175792, 131824,  21078]), array([177616, 169296,  20306]), array([185584, 170240,  21046]), array([197504, 241840,  23068]), array([169008, 203792,  21087]), array([178528,  89504,  21709]), array([180288, 258960,  23956]), array([179792,  97680,  20633]), array([170608, 151712,  21647]), array([170352, 192608,  20580]), array([174720, 152224,  21821]), array([169488, 174880,  20977]), array([179936, 253344,  24349]), array([171712, 189168,  21053]), array([176080, 118192,  21795]), array([19

In [7]:
###define a local path where you would like cells to be downloaded
my_local_path = ''
##loop over all cell ids and root locations to generate and extract skeletons
index = 0
for i, j in zip(oids, roots):
    print (index)
    index +=1
    print(i)
    print (j)
    sk_l2 = pcg_skel.coord_space_meshwork(
            root_id=i,
            client=client,
            root_point=j,
            root_point_resolution=[4, 4, 40],
            collapse_soma=True,
            synapses="all",
            synapse_table="synapses_pni_2",
            timestamp = settime3
            )

    print (sk_l2.root)

    features.add_volumetric_properties(sk_l2, client)
    features.add_segment_properties(sk_l2)
    radius = sk_l2.anno.segment_properties.df['r_eff'].values
    
    axon_inds, Q = meshwork.algorithms.split_axon_by_annotation(sk_l2, pre_anno='pre_syn', post_anno='post_syn')
    sk_l2.anno.add_annotations('is_axon', axon_inds, mask=True)
    
#     sk_l2.export_to_swc(axon_label = 'is_axon', resample_spacing = 1550, radius = radius, soma_label=sk_l2.root, filename = os.path.join(my_local_path, '{}_resample1550.swc'.format(i)))


0
864691135865968005
[179792 179488  20946]
11941
1
864691135940881830
[191120 109792  21792]
2909
2
864691135939924646
[185920 257680  22934]
2557
3
864691135693857727
[196288 247568  24687]
14193
4
864691135490940135
[180752 135200  22612]
9738
5
864691135697251738
[177936 215664  21555]
6674
6
864691135857273902
[181680 160112  21810]
8502
7
864691136334954675
[178176 152192  20167]
815
8
864691135565849111
[189040 118352  21048]
1403
9
864691136912943345
[168512 127088  20650]
1929
10
864691136662432990
[175792 131824  21078]
10232
11
864691135586352764
[177616 169296  20306]
4921
12
864691135545730856
[185584 170240  21046]
3318
13
864691136904366770
[197504 241840  23068]
2167
14
864691135479412678
[169008 203792  21087]
11499
15
864691136723961597
[178528  89504  21709]
857
16
864691136117381284
[180288 258960  23956]
9448
17
864691135518510218
[179792  97680  20633]
1802
18
864691136371168136
[170608 151712  21647]
4994
19
864691136904340658
[170352 192608  20580]
5854
20
86469

10717
163
864691135577202181
[150880 183088  18273]
6116
164
864691135699487522
[234304 183184  24031]
9444
165
864691135013417622
[231104 187856  19274]
6152
166
864691135341516741
[241728 209360  24584]
4431
167
864691135118298333
[238080 196896  19088]
4581
